In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

N=666
num_bins=50

# I. Monte-Carlo for some explicit multiplicative free convolutions.

Recall that the Marchenko-Pastur law is the universal limit for singular values of Gaussian matrices. Following Marchenko and Pastur (1967)
$$ \frac{1}{2 \pi} \frac{\sqrt{(x-l)(r-x)}}{x} dx ,$$
where 
$$r = (1+\sqrt{c})^2$$
$$l = (1-\sqrt{c})^2$$
and $c$ being the scale parameter.

In [ ]:
c = 3 # MP scale parameter
r = (1+np.sqrt(c))**2 #Right end
l = (1-np.sqrt(c))**2 #Left end

G = np.random.normal( size=(N,c*N) )
W = G.dot( G.transpose() )
W = W/N
diag, U = np.linalg.eig(W)
diag = np.sort( diag )

# Histogram of singular values
fig, ax = plt.subplots()
n, bins, patches = ax.hist(diag, num_bins, density=True)
y = np.sqrt( (r-bins)*(bins-l) )/(2*np.pi*bins)
ax.plot(bins, y, '--', linewidth=4)
ax.set_xlabel('Eigenvalues')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of singular values for N={}'.format(N))
fig.tight_layout()
plt.xlim(0,r+0.5)
plt.show()

# II. Cauchy-Stieljes transform: Empirical vs Theoretical

For convenience, let us compute:
$$ G(z) = \int_\mathbb{R} \frac{\mu_{MP}(dt)}{z-t}$$

In [ ]:
def G_theoretical(z):
    sqrt = np.sqrt( (z-c-1)*(z-c-1)-4*c )
    positive_imag = (np.imag(sqrt)>0)
    sqrt = positive_imag*sqrt - (1-positive_imag)*sqrt
    #res = (z+c-1-sqrt)/(2*c*z)-2/(c*z)
    #return res*c
    return (z-c+1-sqrt)/(2*z)

def G_empirical(z):
    array = z[...,None]-diag[...,:]
    return np.sum( 1/array, axis=-1)/len(diag)

def G_prime_empirical(z):
    array = z[...,None]-diag[...,:]
    return np.sum( -1/(array*array), axis=-1)/len(diag)

def G_second_empirical(z):
    array = z[...,None]-diag[...,:]
    return np.sum( 2/(array*array*array), axis=-1)/len(diag)

def M_theoretical(z):
    return z*G_theoretical(z)-1

def M_empirical(z):
    return z*G_empirical(z)-1

def M_prime_empirical(z):
    return z*G_prime_empirical(z) + G_empirical(z)

def M_second_empirical(z):
    return z*G_second_empirical(z) + 2*G_prime_empirical(z)

z0 = np.array( complex(1.0+1.0j) )
print(z0)
print( G_theoretical(z0) )
print( G_empirical(z0) )

### II. 1. Zeroes of the second kind

In [ ]:
def newton_raphson( f, f_prime, initial, max_iter=10, tol=1e-12):
    z = initial
    k = 0
    while k<max_iter:
        value = f(z)
        derivative_value = f_prime(z)
        z = z - value/derivative_value
        if abs(value)<tol:
            break
        k = k + 1
    return z

# f is assumed decreasing between a<b
def dichotomy( a, b, f, max_iter=10, tol=1e-8):
    k = 0
    while k<max_iter:
        c = (a+b)/2
        value = f(c)
        if value>0:
            a = c
        else:
            b = c
        k = k + 1
    return (a+b)/2

zeroes_first_kind  = diag
zeroes_second_kind = np.zeros( shape=(len(diag)-1) )
for i in range( len(diag) - 1):
    z = dichotomy( diag[i], diag[i+1], G_empirical)
    z = newton_raphson( G_empirical, G_prime_empirical, z )
    assert( diag[i] < z )
    assert( z < diag[i+1])
    if( (diag[i] > z) or (diag[i+1] < z) ):
        print("i: ", i)
        print(diag[i], " ", diag[i+1])
        print("z: ", z)
        print("")
    zeroes_second_kind[i] = z
#

# Plotting
plt.figure( figsize=(20,10) )
plt.scatter( np.real( zeroes_first_kind  ), np.imag( zeroes_first_kind  ), c='r', label='First kind')
plt.scatter( np.real( zeroes_second_kind ), np.imag( zeroes_second_kind ), c='b', label='Second kind')
plt.legend()
plt.show()

In [ ]:
def Markov_Krein_empirical(z):
    array1 = z[...,None]-zeroes_first_kind[...,:]
    array2 = z[...,None]-zeroes_second_kind[...,:]
    return 1/z + np.sum( 1/array2, axis=-1) - np.sum( 1/array1, axis=-1)

def Markov_Krein_prime_empirical(z):
    array1 = z[...,None]-zeroes_first_kind[...,:]
    array2 = z[...,None]-zeroes_second_kind[...,:]
    return -1/(z*z) - np.sum( 1/(array2*array2), axis=-1) + np.sum( 1/(array1*array1), axis=-1)

### II. 2. Argument principle for locating branch points

In [ ]:
degree = len(diag)

print("Eigenvalues")
print("min: ", np.min(diag))
print("max: ", np.max(diag))

mesh_size = 10000
radius = np.max(diag)/2 + 1
center = np.max(diag)/2
interval = np.linspace(0, 2*np.pi, mesh_size)
contour = center + radius*( np.cos(interval) + np.sin(interval)*1.0j)
plt.scatter( np.real(diag), np.imag(diag), c='r')
plt.plot( np.real(contour), np.imag(contour) )
plt.show()

def index_integrand(z):
    #values = M_second_empirical(z)/M_prime_empirical(z)
    values = Markov_Krein_prime_empirical(z)/Markov_Krein_empirical(z)
    #values = 1/(contour)
    return values

values = index_integrand(contour)
dz = 1.0j*(contour-center)*2*np.pi/(mesh_size) 
index  = np.sum(dz*values)/(2*np.pi*1.0j)
print( "Index: ", index)
print( "Root count: ", index+2*degree)


In [ ]:
box_corners_enum = [ 'bottom_right',
             'top_right',
             'top_left',
             'bottom_left'
              ]

box_segments_enum = []
for i in range( len(box_corners_enum) ):
       current_corner = box_corners_enum[i]
       next_corner    = box_corners_enum[(i+1) % 4]
       box_segments_enum.append( (current_corner, next_corner) )
print( "Box segments enumeration: ")
print( box_segments_enum )

def compute_index( box, mesh_size, plot=True, color='b'):
       interval =  np.linspace( 0,1, mesh_size)
       integral = 0
       for segment in box_segments_enum:
              vector = box[ segment[1] ] - box[ segment[0] ]
              origin = box[ segment[0] ]
              s = origin + interval*vector
              #
              values = index_integrand(s)
              dz = ( s[-1]-s[0] )/mesh_size
              integral = integral + np.sum( values*dz )
              #
              if plot:
                     x = np.real(s)
                     y = np.imag(s)
                     plt.plot( x, y, c=color)
       return integral/(2*np.pi*1.0j)

def extend_box( box ):
       extended_box = box
       span   = extended_box['top_left']-extended_box['bottom_right']
       height = np.abs( np.imag(span) )
       width  = np.abs( np.real(span) )
       extended_box['top_right']   = extended_box['top_left']     + width
       extended_box['bottom_left'] = extended_box['bottom_right'] - width
       extended_box['height'] = height
       extended_box['width']  = width
       return extended_box

In [ ]:
mesh_size = 10000
radius = 1
box = {
       'top_left'    : -1.0 + radius*1.0j,
       'bottom_right': np.max(diag) + 1 - radius*1.0j,
}
box   = extend_box(box)
index = compute_index( box, mesh_size, plot=True)
index = np.real(index+2*degree)
root_count = np.round( index )
error = index-root_count
print( "Index: ", index)
print( "Root count: ", root_count)

plt.scatter( np.real(diag), np.imag(diag), c='r')
plt.show()


In [ ]:
def plot_box( box, mesh_size, color):
    interval =  np.linspace( 0,1, mesh_size)
    segments = []

    s = box['bottom_right'] + interval*( box['top_right'] - box['bottom_right'] )
    segments.append( s )
    s = box['top_right'] + interval*( box['top_left'] - box['top_right'] )
    segments.append( s )
    s = box['top_left'] + interval*( box['bottom_left'] - box['top_left'] )
    segments.append( s )
    s = box['bottom_left'] + interval*( box['bottom_right'] - box['bottom_left'] )
    segments.append( s )

    for s in segments:
        x = np.real(s)
        y = np.imag(s)
        plt.plot( x, y, c=color)      

    return

mesh_size = 10000

plt.figure( figsize=(20,10) )

# Pass 0: Bounding box
print("Pass 0:")
radius = 1
box = {
       'top_left'    : -1.0 + radius*1.0j,
       'bottom_right': np.max(diag) + 1 - radius*1.0j,
}
box   = extend_box(box)
index = compute_index( box, mesh_size, plot=True)
index = np.real(index+2*degree)
root_count = int( np.round( index ) )
error = index-root_count
print( "Index: ", index)
print( "Root count: ", root_count)
print( "")


root_counter = 0
total_roots  = int( 0.5*root_count ) # Total number of roots in upper half plane
box['bottom_left'] = box['top_left'] # For initialization, bottom_left needs to be the previous top_left

# Loop for multiple passes and more
boxes_with_roots = []
for i in range(13):
       print(f"Pass {i+1}:")
       #print("Height: ", radius)
       radius = radius/2
       box = {
              'top_left'    : box['bottom_left'],
              'bottom_right': np.max(diag) + 1 + radius*1.0j,
       }
       box   = extend_box(box)
       index = compute_index( box, mesh_size, plot=False)
       index = np.real(index)
       root_count = int( np.round( index ) )
       root_counter = root_counter + root_count
       error = index-root_count
       print( "Index: ", index)
       print( "Root count / Total: ", root_count, '/', root_counter)
       print( "Found:", root_counter, " / ", total_roots )
       print( "")
       #
       if root_count>0:
              box['root_count'] = root_count
              boxes_with_roots.append( box )
#

# Plotting
for box in boxes_with_roots:
       color = np.random.rand(3)
       plot_box( box, mesh_size, color)
plt.scatter( np.real(diag), np.imag(diag), c='r')
plt.show()

In [ ]:
def split_box( box ):
    span = box['top_left']-box['bottom_right']
    span = ( abs(np.real(span)), abs(np.imag(span)) )
    if span[0] > span[1]:
        box1 = {
            'top_left'    : box['top_left'],
            'bottom_right': box['bottom_right']-0.5*span[0],
        }
        box2 = {
            'top_left'    : box['top_left']+0.5*span[0],
            'bottom_right': box['bottom_right'],
        }
    else:
        box1 = {
            'top_left'    : box['top_left'],
            'bottom_right': box['bottom_right']+0.5*span[1]*1.0j,
        }
        box2 = {
            'top_left'    : box['top_left']-0.5*span[1]*1.0j,
            'bottom_right': box['bottom_right'],
        }
    #
    box1 = extend_box( box1 )
    box2 = extend_box( box2 )
    return box1, box2

def refine_boxes( coarse_boxes ):
    refined_boxes = []
    for box in coarse_boxes:
        split_boxes = split_box( box )
        #
        for refined_box in split_boxes:
            index = compute_index( refined_box, mesh_size, plot=False)
            index = np.real(index)
            root_count = int( np.round( index ) )
            if root_count>0:
                refined_box['root_count'] = root_count
                refined_boxes.append( refined_box )
    return refined_boxes

refined_boxes = boxes_with_roots
print("Pass / Number of refined boxes / Total number of roots:")
for i in range(15):
    refined_boxes = refine_boxes( refined_boxes )
    counts = [ box['root_count'] for box in refined_boxes]
    print( i+1, ": ", len(refined_boxes), " / ", np.sum(counts) )

In [ ]:
# Plotting
plt.figure( figsize=(20,10) )
for box in refined_boxes:
       color = np.random.rand(3)
       plot_box( box, mesh_size, color)
plt.scatter( np.real(diag), np.imag(diag), marker='*', c='r', label=f'Eigenvalues (N={N})')
plt.title( 'Eigenvalues and estimation of branching points')
plt.legend()
plt.show()

In [ ]:
def box_midpoint( box ):
    return 0.5*( box['top_left'] + box['bottom_right'] )

branch_points = []
for box in refined_boxes:
    midpoint = box_midpoint( box )
    z = newton_raphson( Markov_Krein_empirical, Markov_Krein_prime_empirical, midpoint)
    print( abs(Markov_Krein_empirical(z)) )
    branch_points.append( z )
branch_points = np.array( branch_points )

# Plotting
plt.figure( figsize=(20,10) )
plt.scatter( np.real(branch_points), np.imag(branch_points), marker='*', c='g', label=f'Branch points (N-1={N-1})')
plt.scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
plt.scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
plt.title( 'Eigenvalues and estimation of branching points')
plt.legend()
plt.show()